This a simple interactive tool to create time series plots. There is an option to include forecasts, mostly just for fun.

In [1]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import numpy as np
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.holtwinters import ExponentialSmoothing

# Load your dataset
data = pd.read_csv('data/20241029_year_month_location.csv')

# Ensure date is in the right format for filtering
data['date'] = pd.to_datetime(data[['year', 'month']].assign(day=1))

# Function to create the interactive graphing tool
def create_interactive_tool(data):
    # UI inputs for user selection
    import ipywidgets as widgets
    from IPython.display import display

    # Dropdowns for selecting start and end year and month
    year_options = sorted(data['year'].unique())
    month_options = sorted(data['month'].unique())

    start_year_selector = widgets.Dropdown(options=year_options, description='Start Year')
    start_month_selector = widgets.Dropdown(options=month_options, description='Start Month')
    end_year_selector = widgets.Dropdown(options=year_options, description='End Year')
    end_month_selector = widgets.Dropdown(options=month_options, description='End Month')

    # Location selector
    location_selector = widgets.SelectMultiple(
        options=data['location'].unique(),
        description='Location(s)',
        disabled=False
    )

    # Variable selector for plotting
    variable_selector = widgets.Dropdown(
        options=['total_money', 'total_discount_money', 'total_tip_money', 'transactions', 'identified_customers', 'square_unique_ids'],
        description='Variable'
    )

    # Forecast model selector
    forecast_selector = widgets.Dropdown(
        options=['ARIMA', 'Weighted Moving Average', 'Exponential Smoothing'],
        description='Forecast'
    )

    # Button to trigger plot
    plot_button = widgets.Button(description="Plot Data")

    # Display widgets
    display(start_year_selector, start_month_selector, end_year_selector, end_month_selector,
            location_selector, variable_selector, forecast_selector, plot_button)

    # Plot function triggered by button
    def on_button_click(b):
        # Define the start and end date for filtering
        start_date = pd.to_datetime({'year': [start_year_selector.value], 'month': [start_month_selector.value], 'day': [1]})
        end_date = pd.to_datetime({'year': [end_year_selector.value], 'month': [end_month_selector.value], 'day': [1]})

        # Filter data based on widget selections
        filtered_data = data[(data['date'] >= start_date.iloc[0]) &
                             (data['date'] <= end_date.iloc[0]) &
                             (data['location'].isin(location_selector.value))]

        # Select variable for plotting
        selected_var = variable_selector.value

        # Group by date to aggregate the selected variable across locations
        time_series_data = filtered_data.groupby('date')[selected_var].sum().reset_index()

        # Plot the data
        fig = px.line(time_series_data, x='date', y=selected_var, title=f'Time Series Plot of {selected_var}')

        # Add Forecast Based on Selection
        if forecast_selector.value == 'ARIMA':
            # ARIMA Forecast
            model = ARIMA(time_series_data[selected_var], order=(1, 1, 1))  # Adjust order as needed
            model_fit = model.fit()
            forecast = model_fit.forecast(steps=12)  # Forecasting next 12 periods

            # Add forecast to plot
            forecast_dates = pd.date_range(time_series_data['date'].iloc[-1], periods=12, freq='MS')
            fig.add_trace(go.Scatter(x=forecast_dates, y=forecast, mode='lines', name='ARIMA Forecast'))

        elif forecast_selector.value == 'Weighted Moving Average':
            # Weighted Moving Average Forecast
            weights = np.arange(1, 13)  # 12-period weighted moving average
            wma_forecast = time_series_data[selected_var].rolling(window=12).apply(lambda x: np.dot(x, weights) / weights.sum(), raw=True)

            # Add forecast to plot
            fig.add_trace(go.Scatter(x=time_series_data['date'], y=wma_forecast, mode='lines', name='Weighted Moving Average Forecast'))

        elif forecast_selector.value == 'Exponential Smoothing':
            # Exponential Smoothing Forecast
            model = ExponentialSmoothing(time_series_data[selected_var], seasonal='add', seasonal_periods=12)
            model_fit = model.fit()
            forecast = model_fit.forecast(steps=12)

            # Add forecast to plot
            forecast_dates = pd.date_range(time_series_data['date'].iloc[-1], periods=12, freq='MS')
            fig.add_trace(go.Scatter(x=forecast_dates, y=forecast, mode='lines', name='Exponential Smoothing Forecast'))

        # Show the plot
        fig.show()

    # Attach the plot function to button click
    plot_button.on_click(on_button_click)

# Run the interactive tool
create_interactive_tool(data)


Dropdown(description='Start Year', options=(2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023, 2024), value…

Dropdown(description='Start Month', options=(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12), value=1)

Dropdown(description='End Year', options=(2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023, 2024), value=2…

Dropdown(description='End Month', options=(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12), value=1)

SelectMultiple(description='Location(s)', options=('The Dram Shop Front St.', 'The Dram Shop Central', 'Cateri…

Dropdown(description='Variable', options=('total_money', 'total_discount_money', 'total_tip_money', 'transacti…

Dropdown(description='Forecast', options=('ARIMA', 'Weighted Moving Average', 'Exponential Smoothing'), value=…

Button(description='Plot Data', style=ButtonStyle())